In [19]:
import sys
import gc
import os
#base_dir = "drive/My Drive/jp_en_translation/".replace("/",os.sep)
base_dir = "../"
emb_dir = "drive/My Drive/util_data/".replace("/",os.sep)
sys.path.append("/Users/ueki/Desktop/work/jp_en_translation")
sys.path.append("/Users/ueki/Desktop/JP_EN_Transalation")
sys.path.append(base_dir)
sys.path.append("drive/My Drive/jp_en_translation".replace("/",os.sep))

In [20]:
from models.Transformer import Model
from utils.LangEn import LangEn
from utils.LangJa import LangJa
from utils.build_emb import build_en_emb,build_jp_emb
from utils.preprocess import loadLangs
import numpy as np
from keras.utils import np_utils
import matplotlib.pyplot as plt
from keras.preprocessing import sequence
import torch

In [21]:
config={
    "corpus_file":base_dir+"data/sub_title.txt",
    "en_col":"description_en",
    "jp_col":"description_jp",
    "mask_token":0,
    "SOS_token":1,
    "EOS_token":2,
    "UNK_token":3,
    "max_features":8000,
    "MAX_LENGTH":20,
    "train_size":15000,
    "val_size":300,
    "batch_size":128,
    "epochs":1,
    "maxlen_enc":20,
    "maxlen_dec":20,
    "n_hidden":500,
    "input_dim":5000,
    "jp_voc":8000,
    "en_voc":5000,
    "output_dim":8000,
    "emb_dim":300,
    "validation_split":0.01,

    "translate_length":20,
    "en_W2V_FILE" : emb_dir+"GoogleNews-vectors-negative300.bin.gz",
    "jp_W2V_FILE":emb_dir+"ja_data/ja.bin",
    "src":"en",
    "trg":"jp",
    "learning_rate":0.001,
    "n_heads":5,
    "n_layer":4,
    "param_dir":base_dir+"param/test.pth"
}

In [22]:
import pandas as pd

In [23]:
jp = base_dir+"data/birecoded.ja-en.ja.txt"
en = base_dir + "data/birecoded.ja-en.en.txt"
out = base_dir+"data/test.txt"

In [24]:
with open(jp) as f1:
    with open(en) as f2:
        text1 = f1.read()
        text2 = f2.read()

In [31]:
ret=""
cnt=0
for l1,l2 in zip(text1.split("\n") ,text2.split("\n")):
    l1 = _norm_jp(l1.replace("\"",""))
    l2 = _norm_en(l2.replace("\"",""))
    if len(l1)<=2 or len(l2)<=2:
        cnt+=1
    else:
        S = l2+"\t"+l1+"\n"
        ret+=S
with open(out,"w") as f3:
            f3.write(ret)
print(cnt)

27756


In [27]:
import MeCab
import pandas as pd
import unicodedata
import string
import re
# import random


def _unicodeToAscii(s):
    return " ".join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != "Mn")


def _norm_en(s):
    ret = []
    for c in s.split():
        c = _unicodeToAscii(c.lower())
        c = re.sub(r"([.!?])", r" \1", c)
        c = re.sub(r"[^a-zA-Z.!?]+", r" ", c)
        ret.append("".join(c.split()))
    return " ".join(ret)


def _norm_jp(sentence):
    tagger = MeCab.Tagger("-Owakati")
    ret = []
    for word in tagger.parse(sentence).split():
        word = re.sub(r"([、。])", r"\1", word)
        ret.append("".join(word.split()))
    return " ".join(ret)


def _length_filter(data, config):
    return data[data.apply(lambda x:len(x[config["en_col"]].split(" ")), axis=1) < config["MAX_LENGTH"]]


def loadLangs(config, shuffle=False):
    en_col = config["en_col"]
    jp_col = config["jp_col"]
    print("reading lines")
    data = pd.read_csv(config["corpus_file"], sep="\t", names=[en_col, jp_col])
    if shuffle:
        print("shuffling")
        data = data.sample(frac=1, random_state=0).reset_index(drop=True)
    print("pre processing")
    data[en_col] = data[en_col].apply(lambda x: "".join(_norm_en(x)))
    data[jp_col] = data[jp_col].apply(lambda x: "".join(_norm_jp(x)))
    data = data[[en_col, jp_col]]
    print("filter length")
    filtered = _length_filter(data, config)
    return filtered
